In [46]:
import numpy as np
import pandas as pd

In [47]:
df = pd.read_csv('iris.csv')
df.tail()

,sepal_length,sepal_width,petal_length,petal_width,Class
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1
99,5.7,2.8,4.1,1.3,1


In [48]:
features = ['sepal_length','sepal_width','petal_length','petal_width']
classes = [0, 1]
X = df[features]
Y = df.Class

In [49]:
X_train = np.array(X)
Y_train = np.array(Y)

In [50]:
def sigmoid(u):
    return 1. / (1. + np.e ** - u)

In [57]:
def back_propagation(X, labels, m, regularization=True):
    d, n = X.shape
    X = np.vstack((np.ones(n), X)).T
    b = -1 * np.ones((n,3))
    for i in range(n):
        print(labels[i])
        idx = features.index(labels[i])
        b[i, idx] = 1.
        
    W01 = np.random.randn(m, d+1)
    W12 = np.random.randn(3, m)
    
    epoch = 0
    learning_rate = 0.01
    th = 1e-1
    l = .1
    
    while epoch < 10000:
        epoch += 1
        g10 = sigmoid(np.dot(X, W01.T))
        g21 = sigmoid(np.dot(g10, W12.T))
        
        e21 = ((g21 * 2 - 1) - b) * g21 * (1. - g21)
        e10 = np.dot(e21, W12) * g10 * (1. - g10)
        
        if regularization:
            W12 -= learning_rate * (np.dot(e21.T, g10) + (1 * W12))
            W01 -= learning_rate * (np.dot(e10.T, X) + (1 * W01))
        else:
            W12 -= learning_rate * (np.dot(e21.T, g10))
            W01 -= learning_rate * (np.dot(e10.T, X))
        
    return W01, W12


    

In [58]:
def cross_validation(X, labels, K, regularization=True):
    total_accuracy = 0.
    idxs = list(range(150))
    np.random.shuffle(idxs)
    for n in range(5):
        head = 30 * n
        tail = 30 * (n+1)
        test_idxs = idxs[head:tail]
        train_idxs = [idxs[i] for i in range(150) if (i < head or 1 >= tail)]
        
        W01, W12 = back_propagation(X[train_idxs].T, labels[train_idxs], K, regularization)
        g10 = sigmoid(np.dot(np.hstack((np.ones(30, 1)), X[test_idxs])), W01.T)
        g21 = sigmoid(np.dot(g10, W12.T))
        
        b = 0 * np.ones((30, 3))
        for i in range(30):
            idx = classes.index(labels[test_idxs[i]])
            b[i, idx] = 1.
            result = np.round(g21)
            correct_cnt = 0
            for i in range(30):
                if(b[i] == result[i]).all():
                    correct_cnt += 1
                    accuracy = float(correct_cnt) / 30
                    total_accuracy += accuracy
    return total_accuracy / 5.
        

In [60]:
def main():
    X = []
    labels = []
    with open('iris.csv') as f:
        lines = map(lambda l: l.rstrip().split(','), f.readlines())
        for line in lines:
            X.append(map(float, line[:4]))
            labels.append(line[4])
    
    X = np.asarray(X)
    labels = np.asarray(labels)
    
    print("With Regularization")
    for K in range(1, 11):
        accuracy = cross_validation(X, Y, K)
        print('[K = %2d] %f' % (K, accuracy))
    print("Without Regularization")
    for K in range(1, 11):
        accuracy = cross_validation(X, Y, K, False)
        print('[K = %2d] %f' % (K, accuracy))
        
main()

With Regularization


ValueError: not enough values to unpack (expected 2, got 1)